# Ejemplo de precion de la luz en 2024
### En este ejemplo vamos a recopilar la informacion del precio para todo 2024 
### Lo vamos a convertir en un dataframe y lo vamos a guardar en formato CSV (para no gastar querys a la API de REE)
### Recuerda:
El token de acceso es una clave personal que identifica tus peticiones a la API de ESIOS.
Sin él, la API rechaza cualquier petición.

La API espera que el token se proporcione en forma de x-api-key

La API se basa en indicadores, cada dato importante tiene un ID de indicador por ejemplo 1001

Valores de status code:
200	Datos correctos, 400 Parámetros incorrectos, 401 Token incorrecto, 403 Token sin permisos, 404 Indicador inexistente

## Primero hacemos una query de los datos

In [2]:
import requests
import pandas as pd

TOKEN = "294da85af17aff04fda25ec930700cf59bde0ac3d8322d075617fc038c1ff224"

url = "https://api.esios.ree.es/indicators/1001"

params = {
    "start_date": "2024-01-01T00:00:00",    # 1 Enero 2024 hasta
    "end_date": "2024-12-31T23:00:00",      # 31 Diciembre 2024
    "time_trunc": "hour"
}

# CORRECT HEADER FOR ESIOS API
headers = {
    "x-api-key": TOKEN,  # Primary method: Use your token as the x-api-key
    # Alternatively, you can use the Authorization header in this specific format:
    # "Authorization": f'Token token="{TOKEN}"'  # Uncomment this line if 'x-api-key' doesn't work
}

try:
    response = requests.get(url, headers=headers, params=params)
    print(f"Status Code: {response.status_code}")
    
    if response.status_code == 200:
        data = response.json()
        print("Request successful. Data keys:", data.keys())
    else:
        print("Request failed.")
        print("Response:", response.text)  # This may contain more specific error details

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

Status Code: 200
Request successful. Data keys: dict_keys(['indicator'])


## Convertimos los datos en un Dataframe
Una vez lo tenemos , debemos hacer una pequeña limpieza(eliminar horas repetidas)

In [ ]:
values = data["indicator"]["values"]
df = pd.DataFrame(values) 

#Eliminamos las horas repetidas:

# Convertir 'datetime' asegurándonos de que sea string
df['datetime_str'] = df['datetime'].astype(str)

# Extraer solo la parte de fecha-hora (sin milisegundos ni zona horaria)
df['datetime_simple'] = df['datetime_str'].str.extract(r'(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2})')[0]

# Convertir a datetime
df['datetime_simple'] = pd.to_datetime(df['datetime_simple'])

# Agrupar
df_unique = df.groupby('datetime_simple', as_index=False).agg({'value': 'first'})
df_unique = df_unique.rename(columns={'datetime_simple': 'datetime'})
df_unique = df_unique.sort_values('datetime')

print(f"{len(df_unique)} horas únicas obtenidas")
print(df_unique.head(24))

✓ 8783 horas únicas obtenidas
              datetime   value
0  2024-01-01 00:00:00  123.83
1  2024-01-01 01:00:00  111.11
2  2024-01-01 02:00:00  110.77
3  2024-01-01 03:00:00  109.28
4  2024-01-01 04:00:00  110.16
5  2024-01-01 05:00:00  111.47
6  2024-01-01 06:00:00  109.95
7  2024-01-01 07:00:00  106.89
8  2024-01-01 08:00:00  107.31
9  2024-01-01 09:00:00  103.25
10 2024-01-01 10:00:00   76.42
11 2024-01-01 11:00:00   47.60
12 2024-01-01 12:00:00   46.40
13 2024-01-01 13:00:00   44.73
14 2024-01-01 14:00:00   44.17
15 2024-01-01 15:00:00   51.29
16 2024-01-01 16:00:00   68.26
17 2024-01-01 17:00:00   88.60
18 2024-01-01 18:00:00   96.15
19 2024-01-01 19:00:00  103.34
20 2024-01-01 20:00:00  110.10
21 2024-01-01 21:00:00  110.04
22 2024-01-01 22:00:00   94.09
23 2024-01-01 23:00:00   88.48


## Creamos un CSV con el DF
Así no tenemos que hacer querys a la API

In [5]:
df_unique.to_csv("precio_pool_2024.csv", index=False)